## SF Amplifier Solution

The following cell contains the source-follwer amplifier layout solution.  If you evaluate the following cell LVS should pass.

In [5]:
from abs_templates_ec.analog_core import AnalogBase



class AmpSF(AnalogBase):
    """A template of a single transistor with dummies.

    This class is mainly used for transistor characterization or
    design exploration with config views.

    Parameters
    ----------
    temp_db : :class:`bag.layout.template.TemplateDB`
            the template database.
    lib_name : str
        the layout library name.
    params : dict[str, any]
        the parameter values.
    used_names : set[str]
        a set of already used cell names.
    kwargs : dict[str, any]
        dictionary of optional parameters.  See documentation of
        :class:`bag.layout.template.TemplateBase` for details.
    """

    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        AnalogBase.__init__(self, temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        """Returns a dictionary containing parameter descriptions.

        Override this method to return a dictionary from parameter names to descriptions.

        Returns
        -------
        param_info : dict[str, str]
            dictionary from parameter name to description.
        """
        return dict(
            lch='channel length, in meters.',
            w_dict='width dictionary.',
            intent_dict='intent dictionary.',
            fg_dict='number of fingers dictionary.',
            ndum='number of dummies on each side.',
            ptap_w='NMOS substrate width, in meters/number of fins.',
            ntap_w='PMOS substrate width, in meters/number of fins.',
            show_pins='True to draw pin geometries.',
        )

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        lch = self.params['lch']
        w_dict = self.params['w_dict']
        intent_dict = self.params['intent_dict']
        fg_dict = self.params['fg_dict']
        ndum = self.params['ndum']
        ptap_w = self.params['ptap_w']
        ntap_w = self.params['ntap_w']
        show_pins = self.params['show_pins']

        fg_amp = fg_dict['amp']
        fg_bias = fg_dict['bias']

        if fg_bias % 2 != 0 or fg_amp % 2 != 0:
            raise ValueError('fg_bias=%d and fg_amp=%d must all be even.' % (fg_bias, fg_amp))

        fg_half_bias = fg_bias // 2
        fg_half_amp = fg_amp // 2
        fg_half = max(fg_half_bias, fg_half_amp)
        fg_tot = (fg_half + ndum) * 2

        nw_list = [w_dict['bias'], w_dict['amp']]
        nth_list = [intent_dict['bias'], intent_dict['amp']]

        ng_tracks = [1, 3]
        nds_tracks = [1, 1]

        n_orient = ['R0', 'MX']

        self.draw_base(lch, fg_tot, ptap_w, ntap_w, nw_list,
                       nth_list, [], [],
                       ng_tracks=ng_tracks, nds_tracks=nds_tracks,
                       pg_tracks=[], pds_tracks=[],
                       n_orientations=n_orient,
                       )

        if (fg_amp - fg_bias) % 4 == 0:
            s_net, d_net = 'VDD', 'vout'
            aout, aoutb, nsdir, nddir = 'd', 's', 2, 0
        else:
            s_net, d_net = 'vout', 'VDD'
            aout, aoutb, nsdir, nddir = 's', 'd', 0, 2

        bias_col = ndum + fg_half - fg_half_bias
        amp_col = ndum + fg_half - fg_half_amp
        amp_ports = self.draw_mos_conn('nch', 1, amp_col, fg_amp, nsdir, nddir,
                                       s_net=s_net, d_net=d_net)
        bias_ports = self.draw_mos_conn('nch', 0, bias_col, fg_bias, 0, 2,
                                        s_net='', d_net='vout')

        vdd_tid = self.make_track_id('nch', 1, 'g', 0)
        vin_tid = self.make_track_id('nch', 1, 'g', 2)
        vout_tid = self.make_track_id('nch', 0, 'ds', 0)
        vbias_tid = self.make_track_id('nch', 0, 'g', 0)

        vin_warr = self.connect_to_tracks(amp_ports['g'], vin_tid)
        vout_warr = self.connect_to_tracks([amp_ports[aout], bias_ports['d']], vout_tid)
        vbias_warr = self.connect_to_tracks(bias_ports['g'], vbias_tid)
        vdd_warr = self.connect_to_tracks(amp_ports[aoutb], vdd_tid)
        self.connect_to_substrate('ptap', bias_ports['s'])

        vss_warrs, _ = self.fill_dummy()

        self.add_pin('VSS', vss_warrs, show=show_pins)
        self.add_pin('VDD', vdd_warr, show=show_pins)
        self.add_pin('vin', vin_warr, show=show_pins)
        self.add_pin('vout', vout_warr, show=show_pins)
        self.add_pin('vbias', vbias_warr, show=show_pins)

        self._sch_params = dict(
            lch=lch,
            w_dict=w_dict,
            intent_dict=intent_dict,
            fg_dict=fg_dict,
            dum_info=self.get_sch_dummy_info(),
        )


import os

# import bag package
import bag
from bag.io import read_yaml

# import BAG demo Python modules
import xbase_demo.core as demo_core
from xbase_demo.demo_layout.core import AmpSFSoln

# load circuit specifications from file
spec_fname = os.path.join(os.environ['BAG_WORK_DIR'], 'specs_demo/demo.yaml')
top_specs = read_yaml(spec_fname)

# obtain BagProject instance
local_dict = locals()
if 'bprj' in local_dict:
    print('using existing BagProject')
    bprj = local_dict['bprj']
else:
    print('creating BagProject')
    bprj = bag.BagProject()

demo_core.run_flow(bprj, top_specs, 'amp_sf_soln', AmpSF, run_lvs=True, lvs_only=True)

using existing BagProject
computing layout
ext_w0 = 1, ext_wend=7, ytop=2880
ext_w0 = 2, ext_wend=9, ytop=3024
final: ext_w0 = 1, ext_wend=7, ytop=2880
creating layout
layout done
computing AMP_SF schematics
creating AMP_SF schematics
running lvs
Running tasks, Press Ctrl-C to cancel.
lvs passed
lvs log is /users/erichang/projects/bag_gen/BAG2_cds_ff_mpt/pvs_run/lvs_run_dir/DEMO_AMP_SF/AMP_SF/lvsLog_20180906_110128hvrv5244
LVS flow done
